# Spam Filter Based on Naive Bayes
In this project, we are going to build a spam filter based on **naive bayes**.

We know that computer can learn from human knowledge and classify message based on human offered probability values. As a result, our task is to "teach" the computer how to classify messages. To do that, we'll use the **multinomial Naive Bayes** algorithm along with a dataset of **5,572 SMS messages that are already classified by humans**.

## Data Introduction
The data is put together by *Tiago A. Almeida and José María Gómez Hidalgo*, which can be downloaded on *The UCI Machine Learning Repository* website.

In [1]:
import pandas as pd

# Import data using pandas, and naming two columns as: Label and SMS
msgs = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

# Inspect the first five rows
msgs.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
# Find percentage of spam and ham
msgs['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We can tell that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are not spams.

## Training Set & Test Set
Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

* A training set, which we'll use to "train" the computer how to classify messages.
* A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep **80%(4,458 messages)** of our dataset for training, and **20%(1,114 messages)** for testing. We will use our algorithm to filter the test set, and compare it with the original human-made result.

In [3]:
# Randomizing the dataset
random_msgs = msgs.sample(frac=1, random_state=1)

# Calculate the index spliting the dataset
split_index = round(len(random_msgs) * 0.8)

# Split the dataset, and reset index
train = random_msgs[:split_index].copy()
test = random_msgs[split_index:].copy()
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

Find the new percentage of spam and ham for both sets.

In [4]:
# Training set
train['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [5]:
# Test set
test['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The ramdomizing result is satisfying in that the percentage of spam and ham messages in both dataset is similar to the origin dataset.

## Data Cleaning
Data cleaning is always an indispensable steps, and this project is not the exception likewise. Let's review our training set.

In [6]:
train.sample(5, random_state=0)

,Label,SMS
2715,ham,Wait.i will come out.. &lt;#&gt; min:)
775,ham,There generally isn't one. It's an uncountable...
117,ham,Of course ! Don't tease me ... You know I simp...
4156,ham,No you'll just get a headache trying to figure...
639,ham,Pete can you please ring meive hardly gotany c...


### Remove Unwanted Characters
Because our algorithm will base one the number of times each words appear in a message, we are going split the column into single words like the form below:

| Label | secret | money | my |
| ------ | ------ | ------ | ------ |
| spam | 2 | 1 | 0 |
| ham | 0 | 1 | 2 |

To do this, we will first remove all the punctuation, and make all letter lower case.

In [7]:
# replace any character that are not word.
train['SMS'] = train['SMS'].str.replace('\W', ' ')

# Convert letter to lower case
train['SMS'] = train['SMS'].str.lower()

train.sample(5, random_state=0)

,Label,SMS
2715,ham,wait i will come out lt gt min
775,ham,there generally isn t one it s an uncountable...
117,ham,of course don t tease me you know i simp...
4156,ham,no you ll just get a headache trying to figure...
639,ham,pete can you please ring meive hardly gotany c...


### Create Vocabulary
Next, we will extract all **unique** word in our training set to form a **vocabulary**.

In [8]:
vocabulary = []

# Add each words in a row to our vocabulary list
def add_word(elem):
    word_list = elem.split()
    for word in word_list:
        vocabulary.append(word)
        
# Apply function to training set
train['SMS'].apply(add_word)

# Removing duplicates by converting list to set
vocabulary = list(set(vocabulary))

# Inspect length of vocabulary
len(vocabulary)

7783

### Reformating the Dataset
Finally, we will use the vocabulary we acquire above to reformat the training set.

In [9]:
# Initializing an empty dictionary like: {secret: [0,0, .... ,0]}
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

# Iterate over the SMS column
# For each word in each row, plus 1 to dictionary with corresponding word and index
train['SMS'] = train['SMS'].str.split()
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [10]:
# Convert the result to a dataframe
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [11]:
# Concatenate the origin training set
train_clean = pd.concat([train, word_counts], axis=1)
train_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Building the algorithm
After we are done with data cleaning, we can start writing the algorithm for our spam filter. First, we will review the core therom for the project.

#### Naive Bayes
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, considering the situation that one word only occur in one message category, we will use **Laplace Smoothing** to avoid that.

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

### Calculaing Constants
We begin by calculating the terms that have fixed value for every message, which is as followed:
* P(Spam) and P(Ham)
* N<sub>Spam</sub>, <sub>NHam</sub>, <sub>NVocabulary</sub>

We will calculate them using the training set we have.

In [12]:
# P(Spam): probability that a message is a spam
spams_num = len(train_clean[train_clean['Label'] == 'spam'])
p_spam = spams_num / len(train_clean['Label'])

# P(Ham): probability that a message is not a spam
hams_num = len(train_clean[train_clean['Label'] == 'ham'])
p_ham = hams_num / len(train_clean['Label'])

# N<sub>Spam</sub>: total number of words in spam message
word_per_spam_message = train_clean[train_clean['Label'] == 'spam']['SMS'].apply(len)
n_spam = word_per_spam_message.sum()

# N<sub>Ham</sub>: total number of words in non-spam message
word_per_ham_message = train_clean[train_clean['Label'] == 'ham']['SMS'].apply(len)
n_ham = word_per_ham_message.sum()

# N<sub>Vocabulary</sub>: total number of words in vocabulary
n_vocabulary = len(vocabulary)

print('p_spam: ' + str(p_spam))
print('p_ham: ' + str(p_ham))
print('n_spam: ' + str(n_spam))
print('n_ham: ' + str(n_ham))
print('n_vocabulary: ' + str(n_vocabulary))

p_spam: 0.13458950201884254
p_ham: 0.8654104979811574
n_spam: 15190
n_ham: 57237
n_vocabulary: 7783


### Calculating Parameters
Next, we will calculate values that are unique for every words.
* P(w<sub>i</sub>|Spam): times the word w<sub>i</sub> appear in spam messages.
* P(w<sub>i</sub>|Ham): times the word w<sub>i</sub> appear in non-spam messages.

In [13]:
# Two dictionaries to store two parameters mentioned above
p_spam_dict = {}
p_ham_dict = {}

# Isolate spam & ham messages in training set
train_spam = train_clean[train_clean['Label'] == 'spam']
train_ham = train_clean[train_clean['Label'] == 'ham']

# Iterate over vocabulary
for word in vocabulary:
    n_word_given_spam = train_spam[word].sum()
    p_word_given_spam = (n_word_given_spam + 1) / (n_spam + 1 * n_vocabulary)
    p_spam_dict[word] = p_word_given_spam
    
    n_word_given_ham = train_ham[word].sum()
    p_word_given_ham = (n_word_given_ham + 1) / (n_ham + 1 * n_vocabulary)
    p_ham_dict[word] = p_word_given_ham

### Building Spam Filter Function
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as **input** a new message (w1, w2, ..., wn)
* Calculates **P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)**
* Compares the values of **P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)**, and:
    * If P(Ham|w1, w2, ..., wn) **>** P(Spam|w1, w2, ..., wn), then the message is classified as **ham**.
    * If P(Ham|w1, w2, ..., wn) **<** P(Spam|w1, w2, ..., wn), then the message is classified as **spam**.
    * If P(Ham|w1, w2, ..., wn) **=** P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [14]:
import re

# Core filter function
def classify(message):
    # Clean the message
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # Initialize two parameters we will use to compare
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Calculate above paramters
    for word in message:
        if word not in vocabulary:
            continue;
        if word in p_spam_dict:
            p_spam_given_message *= p_spam_dict[word]
        if word in p_ham_dict:
            p_ham_given_message *= p_ham_dict[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [15]:
# Roughly test it manually
res1 = classify('Come get your prize using the link below')
res2 = classify('I will take the money from the bank for you')
print(res1)
print(res2)

spam
ham


## Test our Algorithm
Finally, we finished building the spam filter. We will test the accuracy for the algorithm using our **test set**

In [20]:
# Variables for saving results
correct = 0
incorrect = 0

# Using a new row saving our result
test['predict_label'] = test['SMS'].apply(classify)

# Exploring the result
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predict_label']:
        correct += 1
    else:
        incorrect += 1
        
# Calculate accuracy using percentage
accuracy = round((correct / test.shape[0]) * 100, 2)

# Print the result
print('Correct prediction: ' + str(correct))
print('Incorrect prediction: ' + str(incorrect))
print('Accuracy: ' + str(accuracy) + '%')

Correct prediction: 1100
Incorrect prediction: 14
Accuracy: 98.74%


## Conclusion
From the result we have, the accuracy of this spam filter algorithm is about 98.74%, which is a satisfying result.

In [23]:
# Explore the incorrect prediction
test[test['Label'] != test['predict_label']]

,Label,SMS,predict_label
114,spam,Not heard from U4 a while. Call me now am here...,ham
135,spam,More people are dogging in your area now. Call...,ham
152,ham,Unlimited texts. Limited minutes.,spam
159,ham,26th OF JULY,spam
284,ham,Nokia phone is lovly..,spam
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification
302,ham,No calls..messages..missed calls,spam
319,ham,We have sent JD for Customer Service cum Accou...,spam
504,spam,Oh my god! I've found your number again! I'm s...,ham
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham
